In [1]:
!pip install transformers

## Pipelin

In [1]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0


In [22]:
sentence_pos = "I love dogs"
sentence_neg = "I swimmingly hate dogs"

In [4]:
classifier(sentence_pos), classifier(sentence_neg)

([{'label': 'POSITIVE', 'score': 0.999713122844696}],
 [{'label': 'NEGATIVE', 'score': 0.9954286813735962}])

## Manual

In [31]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, BertTokenizer

model_name = 'distilbert-base-uncased-finetuned-sst-2-english'
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)
#

In [7]:
distil_tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [8]:
manual_distil_tokenizer = DistilBertTokenizer.from_pretrained(model_name)

In [12]:
outputs = distil_tokenizer(sentence_pos)
print(sentence_pos)
print(outputs)
print(distil_tokenizer("I love dogs and cats"))

I love dogs
{'input_ids': [101, 1045, 2293, 6077, 102], 'attention_mask': [1, 1, 1, 1, 1]}
{'input_ids': [101, 1045, 2293, 6077, 1998, 8870, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}


In [13]:
manual_distil_tokenizer_from_bert = BertTokenizer.from_pretrained(model_name)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [23]:
inputs_pos, inputs_neg = distil_tokenizer(sentence_pos), distil_tokenizer(sentence_neg)
inputs_pos, inputs_neg

({'input_ids': [101, 1045, 2293, 6077, 102], 'attention_mask': [1, 1, 1, 1, 1]},
 {'input_ids': [101, 1045, 5742, 2135, 5223, 6077, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]})

In [21]:
distil_tokenizer.convert_ids_to_tokens([1045, 7078, 5223, 6077])

['i', 'absolutely', 'hate', 'dogs']

In [24]:
distil_tokenizer.convert_ids_to_tokens([ 1045, 5742, 2135, 5223, 6077])

['i', 'swimming', '##ly', 'hate', 'dogs']

In [33]:
pt_inputs = distil_tokenizer(
    [sentence_pos, sentence_neg],
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

print(f"{sentence_pos}\n{pt_inputs['input_ids'][0].numpy()}")
print(f"{sentence_neg}\n{pt_inputs['input_ids'][1].numpy()}")
print(f"{sentence_pos}\n{pt_inputs['attention_mask'][0].numpy()}")
print(f"{sentence_neg}\n{pt_inputs['attention_mask'][1].numpy()}")

I love dogs
[ 101 1045 2293 6077  102    0    0]
I swimmingly hate dogs
[ 101 1045 5742 2135 5223 6077  102]
I love dogs
[1 1 1 1 1 0 0]
I swimmingly hate dogs
[1 1 1 1 1 1 1]


In [35]:
pt_outputs = pt_model(**pt_inputs)
pt_outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-3.9494,  4.2067],
        [ 2.3072, -1.9546]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [36]:
import torch.nn as nn
pt_predictions = nn.functional.softmax(pt_outputs.logits, dim=-1)
pt_predictions

tensor([[2.8688e-04, 9.9971e-01],
        [9.8610e-01, 1.3901e-02]], grad_fn=<SoftmaxBackward0>)

In [37]:
pt_predictions.argmax(dim=-1)

tensor([1, 0])